In [36]:
import psycopg2
import pandas as pd
from plotly.offline import init_notebook_mode, iplot
from plotly.offline import plot 
import plotly.graph_objects as go
import warnings
import datetime
warnings.filterwarnings('ignore')

# import q_credentials.db_secmaster_cloud_cred as db_secmaster_cred
import q_credentials.db_secmaster_cred as db_secmaster_cred
import q_credentials.db_indicator_cred as db_indicator_cred

import dropbox
import pandas as pd



In [7]:
# connect to our securities_master database
conn_secmaster = psycopg2.connect(host=db_secmaster_cred.dbHost , database=db_secmaster_cred.dbName, user=db_secmaster_cred.dbUser, password=db_secmaster_cred.dbPWD)

In [8]:
conn_indicator = psycopg2.connect(host=db_indicator_cred.dbHost , database=db_indicator_cred.dbName, user=db_indicator_cred.dbUser, password=db_indicator_cred.dbPWD)

In [12]:
selected_instrument='us_equity'
sql="""SELECT max(s.id) as id, s.ticker, s.name, max(d.date_price) as date
FROM symbol s inner join d_data d on s.id= d.symbol_id where s.instrument='{}'
group by s.ticker,s.name""".format(selected_instrument)
df_symbols=pd.read_sql(sql,con=conn_indicator)
df_symbols

,id,ticker,name,date
0,2,AAME,Atlantic American Corporation Common Stock,2020-11-06
1,6,AAN,"Aaron''s Holdings Company, Inc.",2020-11-06
2,10,AAON,"AAON, Inc. Common Stock",2020-11-06
3,1158,AAPL,Apple Inc. Common Stock,2020-11-06
4,26,AB,"AllianceBernstein Holding, L.P.",2020-11-06
...,...,...,...,...
1156,1146,YRCW,"YRC Worldwide, Inc. Common Stock",2020-11-06
1157,1149,ZBRA,Zebra Technologies Corporation Class A Common ...,2020-11-06
1158,1152,ZEUS,"Olympic Steel, Inc. Common Stock",2020-11-06
1159,1156,ZION,Zions Bancorporation N.A. Common Stock,2020-11-06


In [34]:
sql="""SELECT
s.ticker, s.name, d.date_price as Date, d.value
FROM
d_data d
INNER JOIN symbol s ON s.id= d.symbol_id
INNER JOIN indicator i ON i.id= d.indicator_id
where i.name='{}'
""".format('alpa_ind')

df_indicator=pd.read_sql(sql,con=conn_indicator)
df_indicator.rename({'date':'Date'},axis=1,inplace=True)
df_indicator=pd.concat([df_indicator.drop(['value'], axis=1), df_indicator['value'].apply(pd.Series)], axis=1)
df_pivoted=df_indicator.pivot(index='Date', columns='ticker', values='level')
df_pivoted

ticker,AAPL,MSFT
Date,,
2016-01-06,0.0,0.0
2016-01-07,0.0,0.0
2016-01-08,0.0,0.0
2016-01-11,0.0,0.0
2016-01-12,0.0,0.0
...,...,...
2020-11-02,0.0,0.0
2020-11-03,0.0,0.0
2020-11-04,0.0,0.0


In [41]:
df_pivoted.to_csv('qc.csv')

In [35]:
!pip install dropbox

     |████████████████████████████████| 681 kB 2.5 MB/s eta 0:00:01


In [39]:
token='TJw42dNCCe0AAAAAAAAAAePjRG5na3V5nAq0RjrkvoH8JRwFkBj4-J8yK0SShTve'

In [42]:
def to_dropbox(dataframe, path, token):

    dbx = dropbox.Dropbox(token)

    df_string = dataframe.to_csv(index=True)
    db_bytes = bytes(df_string, 'utf8')
    dbx.files_upload(
        f=db_bytes,
        path=path,
        mode=dropbox.files.WriteMode.overwrite
    )

In [43]:
to_dropbox(dataframe=df_pivoted,path='/test/alpha_pattern_v1.csv',token=token)